In [1]:
import slamBase as sb
import cv2
import numpy as np

In [2]:
# Load Images to List
imgPath = '/home/lin/RGBD_slam/data/temp/'
cameraDataPath = '/home/lin/RGBD_slam/data/kinect.yml'


In [7]:
rgbList = []
depthList = []

for i in range(1,6):
    rgbList.append( cv2.imread(imgPath + 'rgb{}.png'.format(i)) )
    depthList.append( cv2.imread(imgPath + 'depth{}.png'.format(i),cv2.COLOR_BGR2GRAY))

In [8]:
cv2.imshow('test', rgbList[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
camera = sb.Camera(cameraDataPath)
pnp = sb.SolvePnP(camera)

#sb.imageToPointCloud(rgbList[4],depthList[4],camera,cloudFileName = imgPath + 'milkTea.pcd')


In [10]:
frameList = []
for i in xrange(len(rgbList)):
    frameList.append(sb.FramePC(rgbList[i], depthList[i], camera))

In [11]:
print len(frameList)

5


In [12]:
TM = []
for i in xrange(len(frameList) - 1):
    TM_ = pnp.solveFrame(frameList[i+1], frameList[i])
    TM.append(TM_)

good matches = 154
solving PnP
good matches = 144
solving PnP
good matches = 77
solving PnP
good matches = 30
solving PnP


In [13]:
PC_joined = frameList[0].pointCloud
for i in range(len(frameList) - 1):
    T = np.identity(4)
    for l in range(i+1):
        T = np.dot(T, TM[l])
    PC_transformed = sb.transformPC(frameList[i + 1].pointCloud, TransformMatrix=T)
    PC_joined = np.concatenate((PC_joined, PC_transformed),axis=0)
    

[[ 0.9865636  -0.12261025  0.10797678 -0.05134284]
 [ 0.11083561  0.98783983  0.10903184 -0.02530667]
 [-0.12003218 -0.09559918  0.9881564   0.02709046]
 [ 0.          0.          0.          1.        ]]
[[ 0.95066495 -0.23146569  0.20654245 -0.09708683]
 [ 0.19175045  0.96181647  0.19529683 -0.05400729]
 [-0.24386045 -0.14605724  0.95874885  0.06267331]
 [ 0.          0.          0.          1.        ]]
[[ 0.95066495 -0.23146569  0.20654245 -0.09708683]
 [ 0.19175045  0.96181647  0.19529683 -0.05400729]
 [-0.24386045 -0.14605724  0.95874885  0.06267331]
 [ 0.          0.          0.          1.        ]]
[[-0.61548909  0.78644043  0.05181341 -0.10576314]
 [-0.42047332 -0.27204753 -0.86555897  0.41679557]
 [-0.66661486 -0.55452826  0.4981195  -0.31484991]
 [ 0.          0.          0.          1.        ]]


In [14]:
print PC_joined.shape

(173959, 3)


In [15]:
colors_joined = []
for i in xrange(len(frameList)):
    colors_joined += frameList[i].colors

In [16]:
sb.savePC(PC_joined,imgPath + 'yanhe.pcd',colors_joined)

PCD file Saved
